In [55]:
import pandas as pd
from textblob import TextBlob
from textblob import Word
from gensim import corpora, models, similarities
import os
import tempfile
import gensim
from sklearn.linear_model import LinearRegression
import math
import numpy as np

TEMP_FOLDER = tempfile.gettempdir()

In [56]:
links = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links2.csv', header = None)
text = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext.csv', header = None)

In [57]:
links2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/links3.csv', header = None)
text2 = pd.read_csv('/Users/scottgleave/Downloads/DataMiningCourse/finalprojectdata/reviewtext2.csv', header = None)

In [58]:
#prep 2nd set of data, which has lots of links that didn't work

links2[5] = text2
links2.drop_duplicates([5], inplace = True)
links2 = links2[links2[5] != '[]']
links2 = links2[links2[5] != '[]']
links2.dropna(subset = [5],inplace = True)

In [59]:
links[5] = text

In [60]:
links.drop_duplicates([5], inplace = True)


In [61]:
links.shape

(1618, 6)

In [62]:
text = links[5]
text.shape

(1618,)

In [63]:
#lowercase text
text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
text.head()

0    ["with all of the development muscle behind ea...
1    ['when i first reviewed civilization vi back i...
2    [' ', 'there aren’t many games that make me fe...
5    ['overkill’s ', ' is an earnest attempt to del...
6    ["it's often with rose-colored glasses that i ...
Name: 5, dtype: object

In [64]:
#remove brackets
text = text.apply(lambda x: x[1:-1])
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
5    'overkill’s ', ' is an earnest attempt to deli...
6    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [65]:
#remove new line
text = text.replace(r'\\n','', regex=True)
text.head()

0    "with all of the development muscle behind ea ...
1    'when i first reviewed civilization vi back in...
2    ' ', 'there aren’t many games that make me fee...
5    'overkill’s ', ' is an earnest attempt to deli...
6    "it's often with rose-colored glasses that i l...
Name: 5, dtype: object

In [66]:
#remove punc
text = text.str.replace('[^\w\s]','')
text.head()

0    with all of the development muscle behind ea d...
1    when i first reviewed civilization vi back in ...
2      there arent many games that make me feel as ...
5    overkills   is an earnest attempt to deliver a...
6    its often with rosecolored glasses that i look...
Name: 5, dtype: object

In [67]:
#remove stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
5    overkills earnest attempt deliver cooperative ...
6    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [68]:
freq = pd.Series(' '.join(text[0]).split()).value_counts()[-5:]
#remove uncommon words
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
5    overkills earnest attempt deliver cooperative ...
6    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [69]:
###remove common words###
freq = pd.Series(' '.join(text[0]).split()).value_counts()[:10]
freq = list(freq.index)
text = text.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Visualize
text.head()

0    development muscle behind ea dices massive sho...
1    first reviewed civilization vi back 2016 award...
2    arent many games make feel cool warframe im bu...
5    overkills earnest attempt deliver cooperative ...
6    often rosecolored glasses look back pokemons k...
Name: 5, dtype: object

In [70]:
text

0       development muscle behind ea dices massive sho...
1       first reviewed civilization vi back 2016 award...
2       arent many games make feel cool warframe im bu...
5       overkills earnest attempt deliver cooperative ...
6       often rosecolored glasses look back pokemons k...
8       toys bob spyro dragon skylanders developer rek...
10      often singleplayer campaign primarily multipla...
11      may played many versions tetris youve probably...
12      keep typical sequel expectations check probabl...
15      déracinéxa0is haunting play invisible ghostlik...
16      whereas normally review would try get heart ma...
17      something reversal usual result games use live...
18      doesnt take long dark intriguing detective sto...
19      every sixyearold game feel fresh modern new pl...
20      sprawling western tale loyalty conviction pric...
22      first postgame dlc episode marvels spiderman h...
23      hero academia one best shonen anime currently ...
24      someth

In [71]:
###Spelling correction###

text[:5].apply(lambda x: str(TextBlob(x).correct()))

###Tokenization (dividing the text into a sequence of words or sentences)###
TextBlob(text[0][1]).words

#Stemming (removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach)
#from nltk.stem import PorterStemmer
#st = PorterStemmer()
#train['text'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

#Lemmatization (more effective option than stemming because 
# converts the word into its root word, rather than just stripping the suffices)
#*Preferred to stemming*#


text = text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text.head()

0    development muscle behind ea dice massive shoo...
1    first reviewed civilization vi back 2016 award...
2    arent many game make feel cool warframe im bul...
5    overkill earnest attempt deliver cooperative a...
6    often rosecolored glass look back pokemons kan...
Name: 5, dtype: object

In [72]:
text = text.to_frame()
text['score'] = links[1]

In [73]:
text.head()

,5,score
0,development muscle behind ea dice massive shoo...,7.5
1,first reviewed civilization vi back 2016 award...,9.4
2,arent many game make feel cool warframe im bul...,8.6
5,overkill earnest attempt deliver cooperative a...,5.3
6,often rosecolored glass look back pokemons kan...,8.3


In [74]:
#train val split

trainval = text.sample(400)
train = text.loc[~text.index.isin(trainval.index), :]
y = train['score']
valy = trainval['score']
trainval.drop(['score'], axis = 1, inplace = True)
train.drop(['score'], axis = 1, inplace = True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [75]:
#for train values
textlist = train.values.tolist()

In [76]:
textlist = [str(word) for word in textlist]

In [77]:
texts = [[word for word in document.lower().split() ]
         for document in textlist]

In [78]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=0.6, keep_n=35000)

In [79]:
dictionary.save(os.path.join(TEMP_FOLDER, 'igntfidf.dict'))

In [80]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'igntfidf.mm'), corpus)  # store to disk, for later use

In [81]:
tfidf = models.TfidfModel(corpus)

In [82]:
corpus_tfidf = tfidf[corpus]

In [83]:
all_topics_csr = gensim.matutils.corpus2csc(corpus_tfidf)

In [84]:
all_topics_csr.shape

(10544, 1218)

In [85]:
all_topics_numpy = all_topics_csr.T.toarray()

In [86]:
all_topics_numpy

array([[0.04099152, 0.01697067, 0.04671337, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [87]:
#validation array
vallist = trainval.values.tolist()
vallist = [str(word) for word in vallist]
valtexts = [[word for word in document.lower().split() ]
         for document in vallist]
valcorpus = [dictionary.doc2bow(valtext) for valtext in valtexts]
testcorpus_tfidf = tfidf[valcorpus]
all_topics_csr2 = gensim.matutils.corpus2csc(testcorpus_tfidf)
all_topics_numpytest = all_topics_csr2.T.toarray()

In [88]:
yarray = y.values
lm = LinearRegression()
lm.fit(all_topics_numpy,yarray)
predictions = lm.predict(all_topics_numpytest)
predictionsdf = pd.DataFrame(predictions)

In [89]:
predictionsdf.describe()

,0
count,400.000000
mean,7.554720
std,0.903092
min,4.854488
25%,7.000564
50%,7.588440
75%,8.213230
max,9.584998


In [90]:
valy = valy.reset_index()
valy

,index,score
0,381,8.0
1,1761,6.3
2,1566,8.2
3,2056,9.0
4,185,8.0
5,2297,6.7
6,316,7.8
7,1394,7.0
8,209,4.0
9,1927,6.6


In [91]:
valy.drop(['index'], axis = 1, inplace = True)
valy

,score
0,8.0
1,6.3
2,8.2
3,9.0
4,8.0
5,6.7
6,7.8
7,7.0
8,4.0
9,6.6


In [92]:
predictionsdf['true'] = valy
predictionsdf

,0,true
0,7.748667,8.0
1,7.595313,6.3
2,6.933918,8.2
3,7.731422,9.0
4,8.361031,8.0
5,7.752362,6.7
6,6.362238,7.8
7,7.576976,7.0
8,5.734731,4.0
9,6.545367,6.6


In [93]:
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf

,0,true,diff
0,7.748667,8.0,0.251333
1,7.595313,6.3,-1.295313
2,6.933918,8.2,1.266082
3,7.731422,9.0,1.268578
4,8.361031,8.0,-0.361031
5,7.752362,6.7,-1.052362
6,6.362238,7.8,1.437762
7,7.576976,7.0,-0.576976
8,5.734731,4.0,-1.734731
9,6.545367,6.6,0.054633


In [94]:
#let's also plot histogram of scores




In [95]:
#residual plot here




In [96]:
#RMSE
predictionsdf['diff'] = predictionsdf['true'] - predictionsdf[0]
predictionsdf['diff'] = predictionsdf['diff']**2
diff = predictionsdf['diff'].values
diff = np.mean(diff)
diff = math.sqrt(diff)
diff

#diff = predictionsdf['diff'].values.tolist()
#diff = math.sqrt(np.mean(diff**2))
#diff

1.0023592115327826

In [97]:
arravg = np.mean(yarray)
predictionsdf['avg'] = arravg
predictionsdf['avgdiff'] = predictionsdf['avg'] - predictionsdf['true']
predictionsdf['avgdiff'] = predictionsdf['avgdiff']**2
avgdiff = predictionsdf['avgdiff'].values
avgdiff = np.mean(avgdiff)
avgdiff = math.sqrt(avgdiff)
avgdiff

1.4636373397131892